# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
data = get_date_list(data)
data = read_data(data)

Record num : 5929


In [ ]:
for data['i'] in tqdm(range(0, data['df_len'])):
    
    data['df']['DateTime_frmt'][data['i']] = dt.datetime.strptime(data['df']['DateTime'][data['i']].split(".")[0],"%Y%m%d %H:%M:%S")
    
    data['ask'] = data['df']['Ask'][data['i']]
    data['bid'] = data['df']['Bid'][data['i']]  
    
    data['tick'] = (data['df']['Ask'][data['i']] + data['df']['Bid'][data['i']])/2
    data['df']['tick'][data['i']] = data['tick']
        
    # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------
    
        # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------


    # lema before after loops --------------------------------
    if len(data['lema_list']) < data['lema_len']:
        data = before_lema(data)        
        continue

    if len(data['lema_list']) == data['lema_len']:
        data = after_lema(data)                  
    # ----------------------------------------------------------
    
    data = get_ohlc(data)    
    
    data = get_position(data)
    if not data['position']:
        continue
        
    # Get Dirs --------------------------------
    if len(data['dir_list']) < 2:
        data['dir_list'].append(data['position'])   
        continue

    elif len(data['dir_list']) == 2:
        data = get_cross_dir(data)
    # ----------------------------------------------------------  
    
    data = get_trend_lines(data)
    data = trend_angle_check(data)

 31%|███████████▌                         | 1858/5929 [00:05<00:18, 225.16it/s]

In [ ]:
data['df']

In [ ]:
num_rec = len(data["df"])
hours = np.round((list(data['df']['DateTime_frmt'])[-1] - list(data['df']['DateTime_frmt'])[0]).total_seconds() / 60 / 60 , 2)
recsphour = int(num_rec/hours)

print(f'Records   : {num_rec}')
print(f'Hours     : {hours}')
print(f'Recs/hour : {recsphour}')
print(f'Recs/min  : {int(recsphour/60)}')
print(f'Recs/sec  : {np.round(recsphour/3600, 1)}')

data['df'].head()

In [ ]:
plot_graph(data)

In [ ]:
data['df'].to_csv('data/temp.csv')

In [ ]:
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)